In [1]:
from comet_ml import Experiment
import torch
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import tqdm
import numpy as np
from loguru import logger
import json
from livelossplot import PlotLosses

In [2]:
from torch import nn
import torch.nn.functional as F

In [3]:
logger.remove()
logger.add("log.txt")

1

In [4]:
DEVICE = "cuda:1"

## Load models for vector generation

## Load data

### Read vectors

In [5]:
embeddings = np.load("data/vectors.npy")

In [41]:
indices = np.arange(embeddings.shape[0]//3)

train_idx, test_idx = train_test_split(indices, test_size=0.3, random_state=42)

train_data = [embeddings[train_idx*3], embeddings[train_idx*3+1], embeddings[train_idx*3+2]]
test_data = [embeddings[test_idx*3], embeddings[test_idx*3+1], embeddings[test_idx*3+2]]

print(train_data[0].shape)
print(test_data[0].shape)

(69982, 768)
(29993, 768)


## Measure cosine similarity differences between pos and neg

In [52]:
def calculate_metrics(method, data, name, show_pbar=True):
    orig_shape = data[0].shape[1]
    pos_similarity = []
    neg_similarity = []

    queue = []
    anchors = method(data[0])
    positives = method(data[1])
    negatives = method(data[2])
   
    for i in tqdm.trange(len(anchors), disable=not show_pbar):
        query, pos, neg = anchors[i], positives[i], negatives[i]
        pos_similarity.append(cosine_similarity([query], [pos])[0][0])
        neg_similarity.append(cosine_similarity([query], [neg])[0][0])

    pos_similarity = np.array(pos_similarity)
    neg_similarity = np.array(neg_similarity)
    
    ratio = np.sum(pos_similarity>neg_similarity)/len(pos_similarity)
    mean_diff = pos_similarity.mean()-neg_similarity.mean()
    mean_pos_sim = pos_similarity.mean()
    
    reduction_rate = orig_shape/len(anchors[0])
    return pd.DataFrame.from_dict([{
                        "ratio": ratio,
                        "mean_pos_sim": mean_pos_sim,
                        "mean_diff": mean_diff,
                        "method": name,
                        "reduction_rate": reduction_rate
                    }])

## Benchmarks

In [53]:
metrics = []

### Baseline

In [54]:
metrics.append(calculate_metrics(lambda x: x, test_data, "CLS vector"))

 10%|███████████████▊                                                                                                                                                  | 2938/29993 [00:01<00:11, 2400.03it/s]


KeyboardInterrupt: 

In [ ]:
# lda_metrics = []
# embeddings = []

# train_data = data.sample(1000, random_state=42)

# for i in tqdm.trange(0, train_data.shape[0], 32):
#     embeddings.extend(embed(

In [40]:
dim = 64

# for idx in range(0, train.shape

In [42]:
for

,query,pos,neg
15773,what does dsm stand for,"In psychology and psychiatry, DSM stands for t...","military mk, what does mark stand for in milit..."
35290,what is social competence in early childhood,Social competence in early childhood is as a s...,The Early Years Learning Framework (EYLF) is d...
358404,stonefield construction phone number,A: The phone number for Stonefield Constructio...,The Building Official reviews applications for...
381806,what book did adam smith write,"Adam Smith FRSA was a Scottish economist, phil...",Was Adam Smith laissez-faire? What did it mean...
244560,how long can you take naltrexone for,You and your doctor have to decide this. Most ...,"Naloxone, also known as Narcan among other nam..."
...,...,...,...
178265,what happens in the first stage of seed germin...,Stage One - Hydration and Metabolism. In the f...,Germination is the process in which the seeds ...
168368,aberdeen airport code,Aberdeen International Airport (Scottish Gaeli...,Welcome to USZip.com. USZip.com USZip.com is a...
314866,how old till baby can sleep on stomach,You may find yourself wondering if your baby c...,Third Month Baby Milestones: Sleep. Your 3-mon...
472372,what are the sisters last names in in the time...,In the Time of the Butterflies Characters. BÃ©...,The nine brightest stars of the Pleiades are n...


### Averaging

In [115]:
def average_vec(dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = x.mean(axis=1).reshape(-1, 1)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [118]:
for rate in [4, 8, 16, 32, 64, 128]:
    method = average_vec(rate)
    metrics.append(calculate_metrics(method, test_data, f"Average parts ({rate})"))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:10<00:00, 2991.96it/s]


In [122]:
pd.concat(metrics).sort_values("ratio").to_csv("results/benchmarks.csv")

### Summing

In [32]:
metrics = []
def average_vec(dim):
    def _inner(embeddings):
        embeddings = embeddings/embeddings.sum(axis=1)[:, None]
        x = embeddings.reshape(-1, dim)
        x = x.sum(axis=1).reshape(-1, 1)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [ ]:
for rate in [4, 8, 16, 32, 64, 128, 256]:
    method = average_vec(rate)
    metrics.append(calculate_metrics(method, test_data, f"Sum parts ({rate})"))

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                | 80616/89978 [00:28<00:03, 2825.38it/s]

In [29]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/sums.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.638589,0.642990,0.212868,Sum parts (256),256.0
0,0.738114,0.704705,0.225757,Sum parts (128),128.0
0,0.819356,0.716323,0.232376,Sum parts (64),64.0
0,0.880449,0.730231,0.231080,Sum parts (32),32.0
0,0.919958,0.733388,0.230101,Sum parts (16),16.0
0,0.944009,0.731226,0.228672,Sum parts (8),8.0
0,0.957645,0.731647,0.229022,Sum parts (4),4.0


### Generic autoencoder

In [67]:
def get_model(input_dim, output_dim):
    encoder = nn.Sequential(
                    nn.Linear(input_dim, 32),
                    nn.ReLU(),
                    nn.Linear(32, output_dim),
            )
    decoder = nn.Sequential(
                    nn.Linear(output_dim, 32),
                    nn.ReLU(),
                    nn.Linear(32, input_dim)
            )
    autoenc = nn.Sequential(
                    encoder,
                    decoder
                )
    return autoenc

def test_autoenc(model, dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, dim)
        x = torch.from_numpy(x).to(torch.float32)
        x = model[0](x).cpu().detach().numpy()
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

In [71]:
data = torch.tensor(train_data).reshape(-1, 768)
train_loader = torch.utils.data.DataLoader(data.to(torch.float32), batch_size=32, shuffle=True)

batch_size = 32

def train_autoenc(model, dim, criterion, epochs=1, params={}):
    optim = torch.optim.Adam(model.parameters(), 3e-4)
    
    experiment = Experiment(
    api_key="sQnInDpH3eOHuFvstNS3jG5jb",
    project_name="vector-compression",
    workspace="wwydmanski",
    display_summary_level=0,
    auto_metric_logging=False
    )
    experiment.add_tag("autoencoder")
    experiment.log_parameters(params)
    
    for epoch in range(1, epochs+1):
        with tqdm.notebook.tqdm(train_loader) as t:
            total_loss = []

            for step, batch in enumerate(t):
                x = batch.reshape(-1, dim)
                batch_loss = []
                for idx in range(0, len(x), batch_size):
                    minibatch = x[idx:idx+batch_size]

                    optim.zero_grad()

                    y_hat = model(minibatch)
                    loss = criterion(minibatch, y_hat)
                    loss.backward()
                    optim.step()

                    batch_loss.append(loss.item())

                experiment.log_metric("batch_loss_", np.mean(batch_loss), step=step, epoch=epoch)

            experiment.log_metric("loss_", np.mean(total_loss), step=step, epoch=epoch)

        res = calculate_metrics(test_autoenc(model, dim), train_data, f"Autoencoder")
        print(res)
        experiment.log_metric("Train accuracy", res["ratio"].values[0], step=epoch*step)
        experiment.log_metric("Train mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
        experiment.log_metric("Train mean difference", res["mean_diff"].values[0], step=epoch*step)
        
    res = calculate_metrics(test_autoenc(model, dim), test_data, f"Autoencoder")
    print(res)
    experiment.log_metric("Test accuracy", res["ratio"].values[0], step=epoch*step)
    experiment.log_metric("Test mean positive similarity", res["mean_pos_sim"].values[0], step=epoch*step)
    experiment.log_metric("Test mean difference", res["mean_diff"].values[0], step=epoch*step)
        
    experiment.log_parameter("Reduction rate", res["reduction_rate"].values[0])
    experiment.end()

In [72]:
metrics = []

In [ ]:
hinge_margin = torch.tensor(0)
gamma = 1
def var_loss(x):
    var = torch.sqrt(x.var(0)+1e-6)
    loss = torch.maximum(hinge_margin, gamma-var).mean()
    return loss


dim=32
output_dim = 8
lr = 3e-4

for input_dim in tqdm.notebook.tqdm([8, 16, 32, 64, 128, 768]):
    for output_dim in tqdm.notebook.tqdm([2, 4, 8, 16, 32, 64, 128]):
        if input_dim <= output_dim:
            continue
        params = {"Input dim": input_dim, "Output dim": output_dim, "LR": lr}

        model = get_model(input_dim, output_dim)

        optim = torch.optim.Adam(model.parameters(), lr)
        criterion_ = nn.CosineSimilarity()
        criterion = lambda x, y: -criterion_(x, y).mean() - 0.1*var_loss(y)

        train_autoenc(model, input_dim, criterion, epochs=4, params=params)
        metrics.append(calculate_metrics(test_autoenc(model, input_dim), test_data, f"Autoencoder ({input_dim}x{output_dim})"))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/vector-compression/1cb349cc926d4652a3c9c7ea158b9903



  0%|          | 0/6561 [00:00<?, ?it/s]

In [124]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/autoencoder.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.751017,0.961556,0.027461,Autoencoder,64.0
0,0.837927,0.960247,0.027499,Autoencoder,32.0
0,0.909922,0.970591,0.023799,Autoencoder,16.0
0,0.935818,0.962036,0.031864,Autoencoder,8.0
0,0.958501,0.987163,0.010953,Autoencoder,2.0


### PCA

In [45]:
metrics = []

In [46]:
from sklearn.decomposition import PCA

In [47]:
def test_pca(model, input_dim):
    def _inner(embeddings):
        x = embeddings.reshape(-1, input_dim)
        x = model.transform(x)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

def get_pca(input_dim, output_dim):
    model = PCA(output_dim)
    # data = train_data[0].reshape(-1, input_dim)
    data = np.concatenate(train_data).reshape(-1, input_dim)
    model.fit(data)
    return model

In [44]:
input_dim = 64
output_dim = 8

model = get_pca(input_dim, output_dim)
calculate_metrics(test_pca(model, input_dim), test_data, f"PCA ({input_dim}x{output_dim})")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29993/29993 [00:10<00:00, 2796.43it/s]


,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.946021,0.735421,0.221545,PCA (64x8),8.0


In [57]:
for input_dim in tqdm.notebook.tqdm([8, 16, 32, 64, 128, 768]):
    for output_dim in tqdm.notebook.tqdm([2, 4, 8, 16, 32, 64, 128]):
        if input_dim <= output_dim:
            continue
            
        model = get_pca(input_dim, output_dim)
        metrics.append(calculate_metrics(test_pca(model, input_dim), test_data, f"PCA ({input_dim}x{output_dim})", show_pbar=False))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [59]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/PCA.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.552829,0.385381,0.106327,PCA (768x2),384.0
0,0.603908,0.583071,0.122124,PCA (768x4),192.0
0,0.679659,0.686105,0.153018,PCA (768x8),96.0
0,0.822058,0.740996,0.202793,PCA (128x2),64.0
0,0.771680,0.742186,0.168080,PCA (768x16),48.0
0,0.882039,0.725696,0.223720,PCA (64x2),32.0
0,0.875238,0.766104,0.187173,PCA (768x32),24.0
0,0.923282,0.732020,0.221876,PCA (64x4),16.0
0,0.935085,0.773679,0.204316,PCA (768x64),12.0
0,0.946387,0.734668,0.223791,PCA (64x8),8.0


### UMAP

In [9]:
metrics = []

In [23]:
from cuml.manifold import UMAP

In [37]:
def get_umap(input_dim, output_dim):
    model = UMAP(n_components=output_dim)
    data = train_data[0][:10000].reshape(-1, input_dim)
    model.fit(data)
    return model

In [38]:
model = get_umap(128, 16)

In [ ]:
for input_dim in [16]:
    for output_dim in [2, 4, 8]:
        if input_dim <= output_dim:
            continue
            
        model = get_umap(input_dim, output_dim)
        metrics.append(calculate_metrics(test_pca(model, input_dim), test_data, f"UMAP ({input_dim}x{output_dim})"))

 60%|█████████████████████████████████████████████████████████████████████████████████████████████████                                                                | 54275/89978 [00:20<00:13, 2678.39it/s]

In [35]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
cmetrics.to_csv("results/UMAP.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.550879,0.668035,0.040766,UMAP (768x2),384.0
0,0.632088,0.911761,0.057221,UMAP (768x4),192.0
0,0.651104,0.957159,0.033607,UMAP (768x8),96.0
0,0.544500,0.846501,0.028204,UMAP (128x2),64.0
0,0.653393,0.968231,0.024588,UMAP (768x16),48.0
0,0.644124,0.954982,0.030882,UMAP (128x4),32.0
0,0.650470,0.975806,0.017704,UMAP (768x32),24.0
0,0.673565,0.965916,0.033695,UMAP (128x8),16.0
0,0.614295,0.824949,0.034135,UMAP (768x64),12.0
0,0.885116,0.629663,0.168973,UMAP (32x4),8.0


### LocallyLinearEmbedding

In [8]:
from sklearn.manifold import LocallyLinearEmbedding

In [63]:
batch_size = 512
def test_lle(model, input_dim):
    def _inner(embeddings):
        res = []
        x = embeddings.reshape(-1, input_dim)
        
        for i in range(0, len(x), batch_size):
            res.append(model.transform(x[i:i+batch_size]))

        x = np.concatenate(res)
        x = x.reshape(len(embeddings), -1)
        return x
    return _inner

def get_lle(input_dim, output_dim):
    model = LocallyLinearEmbedding(n_components=output_dim, n_jobs=20)
    data = train_data[0].reshape(-1, input_dim)[:5000]
    model.fit(data)
    return model

In [64]:
metrics = []

In [65]:
for input_dim in tqdm.notebook.tqdm([8, 16, 32, 64, 128, 768][::-1]):
    for output_dim in tqdm.notebook.tqdm([2, 4, 8, 16, 32, 64, 128]):
        if input_dim <= output_dim:
            continue
            
        model = get_lle(input_dim, output_dim)
        metrics.append(calculate_metrics(test_lle(model, input_dim), test_data, f"LocallyLinearEmbedding ({input_dim}x{output_dim})", show_pbar=False))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [66]:
cmetrics = pd.concat(metrics)

max_ratio = cmetrics.groupby("reduction_rate").max()["ratio"]
cmetrics = cmetrics[cmetrics.apply(lambda x: x["ratio"]==max_ratio[x["reduction_rate"]], axis=1)]
cmetrics = cmetrics.sort_values("reduction_rate", ascending=False)
# cmetrics.to_csv("results/LocallyLinear.csv")

cmetrics

,ratio,mean_pos_sim,mean_diff,method,reduction_rate
0,0.624612,0.796312,0.109099,LocallyLinearEmbedding (768x2),384.0
0,0.658020,0.819372,0.113057,LocallyLinearEmbedding (768x4),192.0
0,0.673090,0.826621,0.110064,LocallyLinearEmbedding (768x8),96.0
0,0.622545,0.653948,0.111262,LocallyLinearEmbedding (128x2),64.0
0,0.688461,0.812073,0.126374,LocallyLinearEmbedding (768x16),48.0
0,0.737639,0.451451,0.179626,LocallyLinearEmbedding (64x2),32.0
0,0.707932,0.776378,0.152379,LocallyLinearEmbedding (768x32),24.0
0,0.803888,0.439338,0.183319,LocallyLinearEmbedding (32x2),16.0
0,0.720935,0.741271,0.172130,LocallyLinearEmbedding (768x64),12.0
0,0.852099,0.424947,0.181779,LocallyLinearEmbedding (32x4),8.0
